In [1]:
from IPython.display import display, HTML

import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib as mlp
import matplotlib.pyplot as plt
import geopandas as gp
import os

%matplotlib inline
plt.style.use('ggplot')

In [2]:
plt.rcParams["figure.facecolor"] = "w"
plt.rcParams["figure.dpi"] = 120
pd.options.display.max_columns = 1000

In [3]:
os.chdir("../../core")
import GT_helper_functions as hf
import GT_load_data as data

In [4]:
base_dir = "../../Outcome Measurement Data/MULTI/"

In [5]:
gbd17 = pd.read_csv("../../Outcome Measurement Data/MULTI/GBD-2017/IHME-GBD_2017_DATA-3f789f48-1.csv")

In [6]:
gbd17.columns

Index(['measure', 'location', 'sex', 'age', 'cause', 'metric', 'year', 'val',
       'upper', 'lower'],
      dtype='object')

In [7]:
#gbd17.age.value_counts()
# All ages :
allages = ["Under 5", "5-14 years", "15-49 years", "50-69 years", "70+ years"]

In [14]:
gbdCounts = gbd17[(gbd17.metric== "Number") & 
      (gbd17.location == "Guatemala") &
      (gbd17.age.isin(allages))]\
    .groupby(["cause", "year", "location", "measure"]).agg({
            "val": "sum", "upper": "sum", "lower": "sum"
        })

In [9]:
gbd15_17pop = pd.read_csv("../../Outcome Measurement Data/MULTI/GBD-2017/IHME_GBD_2017_POP_2015_2017_Y2018M11D08.CSV")
gbd10_14pop = pd.read_csv("../../Outcome Measurement Data/MULTI/GBD-2017/IHME_GBD_2017_POP_2010_2014_Y2018M11D08.CSV")
gbd05_09pop = pd.read_csv("../../Outcome Measurement Data/MULTI/GBD-2017/IHME_GBD_2017_POP_2005_2009_Y2018M11D08.CSV")
gbd00_04pop = pd.read_csv("../../Outcome Measurement Data/MULTI/GBD-2017/IHME_GBD_2017_POP_2000_2004_Y2018M11D08.CSV")

In [10]:
gbd15_17pop.sex_name.value_counts()

Both      187131
Male      187131
Female    187131
Name: sex_name, dtype: int64

In [11]:
def aggGtPop(dataframe):
    return dataframe[dataframe.age_group_name.isin(allages) & 
                (dataframe.location_name == "Guatemala") &
                (dataframe.sex_name == "Both")
               ].groupby("year_id").agg({
        "val": "sum",
        "upper": "sum",
        "lower": "sum"
    })

In [12]:
GtPop = pd.concat([
    aggGtPop(gbd15_17pop),
    aggGtPop(gbd10_14pop),
    aggGtPop(gbd05_09pop),
    aggGtPop(gbd00_04pop)
          ])

In [13]:
GtPop.sort_index()

,val,upper,lower
year_id,,,
2000,1.093944e+07,1.183010e+07,1.010558e+07
2001,1.128742e+07,1.214730e+07,1.046162e+07
2002,1.164147e+07,1.245544e+07,1.083104e+07
2003,1.199778e+07,1.295235e+07,1.110808e+07
2004,1.235307e+07,1.328605e+07,1.139675e+07
2005,1.270121e+07,1.373576e+07,1.162490e+07
2006,1.304212e+07,1.429803e+07,1.184061e+07
2007,1.338471e+07,1.482579e+07,1.203974e+07
2008,1.373260e+07,1.516473e+07,1.220957e+07


In [30]:
gbdCounts = gbdCounts.reset_index()

In [31]:
gbdCounts["cause2"] = gbdCounts.cause.map(lambda x: "TB" if "Tuberculosis" in x else "HIV" if "HIV" in x else "Malaria" if "Malaria" in x else "")

In [58]:
gbdCounts[gbdCounts.year >= 2000].groupby(["year", "cause2", "measure"]).apply(lambda x: 100000*x.val.sum()/GtPop.loc[x["year"].values[0]].val)\
.round(1)

year  cause2   measure   
2000  HIV      Deaths            2.2
               Incidence        20.5
               Prevalence       99.5
      Malaria  Deaths            0.4
               Incidence      2013.1
               Prevalence     1080.8
      TB       Deaths            8.0
               Incidence        25.8
               Prevalence    29944.1
2001  HIV      Deaths            2.4
               Incidence        20.4
               Prevalence      107.4
      Malaria  Deaths            0.3
               Incidence      1920.1
               Prevalence     1040.4
      TB       Deaths            7.2
               Incidence        25.4
               Prevalence    29453.6
2002  HIV      Deaths            3.2
               Incidence        19.6
               Prevalence      118.4
      Malaria  Deaths            0.3
               Incidence      1679.3
               Prevalence      961.8
      TB       Deaths            6.8
               Incidence        25.1
            

In [52]:
GtPop.loc[2000]

val      1.093944e+07
upper    1.183010e+07
lower    1.010558e+07
Name: 2000, dtype: float64